## CS 5330
## Final Project: Garbage Classification
### Group members: Jinghan Gao, Tianhao Zhang, Jialu Bi
This notebook includes codes for preprocessing the data and training the classification model.

#### Read the data, resize the images (256x256), and convert them to tensors

In [1]:
import os
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

dir  = os.path.join("data", "Garbage classification", "Garbage classification")

transformations = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
dataset = ImageFolder(dir, transform=transformations)
dataset_size = len(dataset)

print(f"Size of the dataset: {dataset_size}")
print(f"Classes: {dataset.classes}")

Size of the dataset: 2527
Classes: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


#### Split the dataset

In [2]:
from torch.utils.data import random_split

# Split into 70% train, 15% valid and 15% test
train_size = int(0.7 * dataset_size)
valid_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - valid_size

train, valid, test = random_split(dataset, [train_size, valid_size, test_size])

print(f"Size of training dataset: {train_size}")
print(f"Size of validation dataset: {valid_size}")
print(f"Size of test dataset: {test_size}")

Size of training dataset: 1768
Size of validation dataset: 379
Size of test dataset: 380


#### Load the dataset

In [3]:
from torch.utils.data.dataloader import DataLoader

# Shuffle the train dataset to prevent ordering bias
data_loader_train = DataLoader(train, batch_size=32, shuffle=True, num_workers=12, pin_memory=True)
data_loader_valid = DataLoader(valid, batch_size=64, num_workers=12, pin_memory=True)
data_loader_test = DataLoader(test, batch_size=64, num_workers=12, pin_memory=True)

#### Load a pre-trained MobileNetV3 model

In [4]:
import torch.nn as nn
from torchvision import models

model = models.mobilenet_v3_large(pretrained=True)
    
# Freeze all layers except the last fully connected layer
for param in model.parameters():
    param.requires_grad = False

# Modify the last fully connected layer to match the number of classes we have
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(dataset.classes))

C:\Users\Kevin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Kevin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#### Finetune the model

In [5]:
import torch
import torch.optim as optim
from tqdm import tqdm

# Define hyperparameters
num_epochs = 20
learning_rate = 1e-4
weight_decay = 1e-4

device = torch.device("cuda" if torch.cuda.is_available() else "CPU")

# Move model to the device
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

best_model_wts = model.state_dict()
best_acc = 0.0

# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    
    # Training phase
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    for inputs, labels in tqdm(data_loader_train, desc="Training..."):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)

        # Calculate the loss
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        
        # Calculate accuracy
        _, preds = torch.max(outputs, 1)
        correct_preds += torch.sum(preds == labels).item()
        total_preds += labels.size(0)
    
    epoch_loss = running_loss / len(data_loader_train.dataset)
    epoch_acc = correct_preds / total_preds
    
    print(f"Training loss: {epoch_loss:.4f}, accuracy: {epoch_acc:.4f}")
    
    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader_valid, desc="Validating..."):
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels).item()
            total_preds += labels.size(0)
    
    val_loss /= len(data_loader_valid.dataset)
    val_acc = correct_preds / total_preds
    print(f"Validation loss: {val_loss:.4f}, accuracy: {val_acc:.4f}")
    
    # Copy the model if it's the best so far
    if val_acc > best_acc:
        best_acc = val_acc
        best_model_wts = model.state_dict()

print(f"Best validation accuracy: {best_acc:.4f}")

# Load the best model weights
model.load_state_dict(best_model_wts)

# Save the model
torch.save(model.state_dict(), "classifier.pth")

Epoch 1/20


Training...: 100%|██████████| 56/56 [00:24<00:00,  2.29it/s]


Training loss: 1.7048, accuracy: 0.2896


Validating...: 100%|██████████| 6/6 [00:23<00:00,  3.85s/it]


Validation loss: 1.5624, accuracy: 0.4934
Epoch 2/20


...

Validation loss: 0.5859, accuracy: 0.8311
Epoch 20/20


Training...: 100%|██████████| 56/56 [00:24<00:00,  2.26it/s]


Training loss: 0.5178, accuracy: 0.8479


Validating...: 100%|██████████| 6/6 [00:24<00:00,  4.07s/it]

Validation loss: 0.5763, accuracy: 0.8338
Best validation accuracy: 0.8338


### Evaluate the model on test set

In [6]:
# Helper function for evaluating the model
def evaluate(model, data_loader, criterion):
    # Set the model to eval mode
    model.eval()
    test_loss = 0.0
    correct_preds = 0
    total_preds = 0
    
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc="Testing..."):
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * inputs.size(0)
            
            # Calculate accuracy
            _, preds = torch.max(outputs, 1)
            correct_preds += torch.sum(preds == labels).item()
            total_preds += labels.size(0)
    
    avg_loss = test_loss / len(data_loader.dataset)
    accuracy = correct_preds / total_preds
    print(f"Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.4f}")
    return avg_loss, accuracy

# Run the evaluation on the test set
test_loss, test_accuracy = evaluate(model, data_loader_test, criterion)

Testing...: 100%|██████████| 6/6 [00:23<00:00,  3.90s/it]

Test Loss: 0.5937, Test Accuracy: 0.8053


#### Visualize some cases

In [24]:
import cv2
import random
import numpy as np
from PIL import Image

def visualize_predictions_on_image(model, image_path, device):
    model.eval()  # Set the model to evaluation mode

    with torch.no_grad():
        # Load the image
        image = Image.open(image_path).convert("RGB")
        input_image = transformations(image).unsqueeze(0).to(device)

        # Perform model inference
        outputs = model(input_image)
        _, preds = torch.max(outputs, 1)

        # Prepare the image for display
        image_np = np.array(image)
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)  # Convert to BGR format for OpenCV

        # Display predicted label
        pred_class = dataset.classes[preds.item()]  # Assuming dataset has the classes attribute
        label_text = f"Predicted: {pred_class}"
        
        # Put the text on the image
        cv2.putText(image_np, label_text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        # Display the image
        cv2.imshow("Prediction", image_np)
        cv2.waitKey(0)

    cv2.destroyAllWindows()

# Path to the image
g_class = "cardboard" # Modify class to see how the model performs on other classes
image_path = os.path.join("data", "Garbage classification", "Garbage classification", f"{g_class}", f"{g_class}{random.randint(1, 200)}.jpg")

# Visualize prediction on the image
visualize_predictions_on_image(model, image_path, device)